# imports

In [75]:
# imports
import requests
import pandas as pd
from notion_client import Client
import datetime
import time
import json

# global setup

In [76]:
BLEND_API_KEY = "org_ba2e4ccfb75e56afc088d9804df57d2623542e8bbd3de2c02bfcb0024daa778c1850bba9de94a2d1ec6a69"
CALL_SCRIPT = "6a5a0412-6481-4533-b560-cf72283e956b"
url = "https://api.bland.ai/v1/calls"
GROQ_API_KEY = "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"
groq_url = "https://api.groq.com/openai/v1/chat/completions"


Customer_phone_number = "+972545842444"
System_phone_number = "null"
Interruption_Threshold_in_ms = 100
LLM_temperature = 1

Customer_name = "John"
Home_address = "123 Main St"
Agent_name = "Christin"
Specialist_name = "David"
Agent_voice = "Public - June 2978"

# call setup

In [77]:
# Headers
headers = {
   'Authorization': BLEND_API_KEY,
   "Content-Type": "application/json"
}

# Data
data = {
    "phone_number": Customer_phone_number,
    "task": "",
    "model": "enhanced",
    "language": "en-US",
    "voice": Agent_voice,
    "voice_settings": {},
    "pathway_id": CALL_SCRIPT,
    "pathway_version": "1", # this is for testing, delete for production
    "local_dialing": False,
    "max_duration": "12",
    "answered_by_enabled": False,
    "wait_for_greeting": True,
    "noise_cancellation": True,
    "ignore_button_press": True,
    "record": False,
    "amd": False,
    "interruption_threshold": Interruption_Threshold_in_ms,
    "voicemail_message": "test",
    "temperature": LLM_temperature,
    "transfer_list": {},
    "pronunciation_guide": [],
    "request_data": {
    "customer name": Customer_name,
    "home address": Home_address,
    "agent name": Agent_name,
    },
    "retry": {
    "wait": 720,
    "voicemail_action": "hangup",
    "voicemail_message": ""
    },
    "dynamic_data": [],
    "analysis_schema": {},
    "calendly": {},
    "timezone": "America/New_York"

}

# API request

# Call action

In [78]:

#send the call
send_call_response = json.loads(requests.request("POST", url, json=data, headers=headers).text)
#print status 
print(send_call_response)

#extract call id
call_id = send_call_response["call_id"]

#check if call is active:
call_id_response = json.loads(requests.request("GET", url+"/"+call_id, headers=headers).text)
while call_id_response["completed"] == False:
    time.sleep(3)
    print("Call is still active")   
    call_id_response = json.loads(requests.request("GET", url+"/"+call_id, headers=headers).text)

    
print("completed")


{'status': 'success', 'message': 'Call successfully queued.', 'call_id': '0ffe5197-d348-42df-a69d-657dd5b1ab9a', 'batch_id': None}
Call is still active
Call is still active
Call is still active
Call is still active
completed


In [79]:
call_id_response = json.loads(requests.request("GET", url+"/"+call_id, headers=headers).text)
call_id_response

{'call_id': '0ffe5197-d348-42df-a69d-657dd5b1ab9a',
 'c_id': '0ffe5197-d348-42df-a69d-657dd5b1ab9a',
 'call_length': 0,
 'batch_id': None,
 'to': '+972545842444',
 'from': '+15712237600',
 'completed': True,
 'created_at': '2025-02-10T10:42:33.3+00:00',
 'inbound': False,
 'queue_status': 'complete',
 'max_duration': 12,
 'error_message': None,
 'variables': None,
 'answered_by': None,
 'record': False,
 'recording_url': None,
 'metadata': {},
 'summary': None,
 'price': 0,
 'started_at': '2025-02-10T10:42:33+00:00',
 'local_dialing': False,
 'call_ended_by': None,
 'pathway_logs': None,
 'analysis_schema': None,
 'analysis': None,
 'transferred_to': None,
 'pathway_tags': [],
 'recording_expiration': None,
 'status': 'busy',
 'pathway_id': '6a5a0412-6481-4533-b560-cf72283e956b',
 'concatenated_transcript': '',
 'transcripts': [],
 'corrected_duration': '0',
 'end_at': '2025-02-10T10:42:40.000Z'}

In [ ]:
call_id = '787b32a7-8ea2-4006-8fa2-f44e82b7bd9f'
call_id_response = json.loads(requests.request("GET", url+"/"+call_id, headers=headers).text)
call_id_response

In [80]:
# extract call information
call_id_response = json.loads(requests.request("GET", url+"/"+call_id, headers=headers).text)
call_info_dict = {}
# if call cosutmer is interested
if "Interested" in call_id_response["pathway_tags"]:
    
    # collect time started
    print(call_id_response["created_at"])
    call_info_dict["created_at"] = call_id_response["created_at"]
    
    # collect costumer email
    print(call_id_response["variables"]["email"])
    call_info_dict["email"] = call_id_response["variables"]["email"]
    
    # collect follow up date from transcription
    # Call to Groq API
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "deepseek-r1-distill-llama-70b",
        "messages": [
            {"role": "system", "content": "You are an AI assistant tasked with analyzing call transcripts and suggesting specific follow-up dates, please resonse with specific time and date format, for example: \"February 6, 2025 12:00 AM\", do not add any other text"},
            {"role": "user", "content": f"Based on this call transcript, suggest a specific follow-up date: {call_id_response['concatenated_transcript']}, the time of the call is{call_id_response["created_at"]}"}
        ]
    }
    groq_response = requests.post(groq_url, json=payload, headers=headers)
    follow_up_date = groq_response.json()['choices'][0]['message']['content'].strip().split('\n')[-1]
    
    print(follow_up_date)
    call_info_dict["follow up date"] = follow_up_date
       
       
    # collect call decision   
    print(call_id_response["pathway_tags"])
    call_info_dict["decision"] = call_id_response["pathway_tags"][0]
    
    print(call_id_response["concatenated_transcript"])
    call_info_dict["script"] = call_id_response["concatenated_transcript"]
    
    
    
    
elif not call_id_response["pathway_tags"]:
    #collect Follow Up Date

    # update Next Follow Up Date to 4 hours after the call
    call_info_dict["Next Follow Up Date"] = (pd.to_datetime(call_id_response["created_at"]) + pd.Timedelta(days=7)).strftime('%Y-%m-%d %H:%M:%S')
    

In [81]:
call_info_dict

{'Next Follow Up Date': '2025-02-17 10:42:33'}